<a href="https://colab.research.google.com/github/rennyatwork/CegepSteFoy_IA/blob/main/08_deep_learnin/RNN_Debit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Importation des packages

In [1]:
#Importation des packages
import numpy as np
from numpy import array
from numpy import asarray
from pandas import read_csv
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout

2. Chargement du jeu de données contenant les température, quantité de précipitations et débit de la rivière Saint-Charles pour quelques jours

In [2]:
df = read_csv('https://raw.githubusercontent.com/rennyatwork/CegepSteFoy_IA/main/08_deep_learning/data/Debit_St-Charles.csv')
print(len(df))
df.head(10)

119


,An,Mois,Jour,Temp,Precip,Debit
0,2023,4,13,7.2,0.0,17.41
1,2023,4,14,6.4,0.0,25.37
2,2023,4,15,5.7,0.0,30.38
3,2023,4,16,7.6,0.0,36.74
4,2023,4,17,5.9,11.8,51.50
5,2023,4,18,5.2,0.7,69.64
6,2023,4,19,4.9,1.7,72.08
7,2023,4,20,2.6,0.0,57.16
8,2023,4,21,3.5,0.0,42.24
9,2023,4,22,10.2,0.0,34.46


SECTION 1: Série unidimensionnelle: nous allons commencer par tenter de prédire les débits en fonction uniquement des débits passés

3. Fonction servant à formatter les données d'un séquance unidimensionnelle

In [3]:
def split_sequence(sequence, n_steps):
 X, y = list(), list()
 for i in range(len(sequence)):
 # find the end of this pattern
  end_ix = i + n_steps
 # check if we are beyond the sequence
  if end_ix > len(sequence)-1:
    break
 # gather input and output parts of the pattern
  seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
  X.append(seq_x)
  y.append(seq_y)
 return array(X), array(y)

4. Définir la séquence des débits

In [4]:
raw_seq = np.array(df)
raw_seq = raw_seq[:,5]
raw_seq = raw_seq.astype(np.float32)
print(raw_seq)

[17.41  25.37  30.38  36.74  51.5   69.64  72.08  57.16  42.24  34.46
 37.96  48.31  58.88  53.85  45.15  37.22  31.66  32.68  42.2   63.83
 55.17  38.87  24.79  17.85  15.75  13.19   9.438  8.42   6.156  5.106
  6.626  5.63   4.519  4.226  4.337  3.58   3.411  3.469  6.423  7.25
  5.621  4.085  3.882  3.391  3.082  2.755  2.36   2.055  1.831  1.961
  2.449  2.189  1.857  1.59   1.617  1.908  1.93   1.987  1.525  1.478
  1.261  1.832  2.562  3.053  3.262  3.716  3.651  3.067  2.844  2.336
  1.84   1.528  1.514  1.554  1.146  1.404  2.057  6.54  10.23   7.732
 11.44  13.41   9.968  5.758  3.485  2.562  2.538  3.102 18.4   45.4
 62.69  55.89  53.82  49.78  39.02  39.25  33.63  21.36  13.94   8.779
  9.718 10.73   9.286  7.464  6.291  5.438  5.994  6.074  5.497  5.776
  5.11   3.651  3.636 12.38  15.68  10.12   7.821 31.46  39.06 ]


5. Formatter la séquence des débits pour l'analyse LSTM à l'aide de la fonction split_sequence()

In [5]:
n_steps = 3

X, y = split_sequence(raw_seq, n_steps)

n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

print(X.shape, y.shape)

for i in range(len(X)):
 print(X[i], y[i])

(116, 3, 1) (116,)
[[17.41]
 [25.37]
 [30.38]] 36.74
[[25.37]
 [30.38]
 [36.74]] 51.5
[[30.38]
 [36.74]
 [51.5 ]] 69.64
[[36.74]
 [51.5 ]
 [69.64]] 72.08
[[51.5 ]
 [69.64]
 [72.08]] 57.16
[[69.64]
 [72.08]
 [57.16]] 42.24
[[72.08]
 [57.16]
 [42.24]] 34.46
[[57.16]
 [42.24]
 [34.46]] 37.96
[[42.24]
 [34.46]
 [37.96]] 48.31
[[34.46]
 [37.96]
 [48.31]] 58.88
[[37.96]
 [48.31]
 [58.88]] 53.85
[[48.31]
 [58.88]
 [53.85]] 45.15
[[58.88]
 [53.85]
 [45.15]] 37.22
[[53.85]
 [45.15]
 [37.22]] 31.66
[[45.15]
 [37.22]
 [31.66]] 32.68
[[37.22]
 [31.66]
 [32.68]] 42.2
[[31.66]
 [32.68]
 [42.2 ]] 63.83
[[32.68]
 [42.2 ]
 [63.83]] 55.17
[[42.2 ]
 [63.83]
 [55.17]] 38.87
[[63.83]
 [55.17]
 [38.87]] 24.79
[[55.17]
 [38.87]
 [24.79]] 17.85
[[38.87]
 [24.79]
 [17.85]] 15.75
[[24.79]
 [17.85]
 [15.75]] 13.19
[[17.85]
 [15.75]
 [13.19]] 9.438
[[15.75 ]
 [13.19 ]
 [ 9.438]] 8.42
[[13.19 ]
 [ 9.438]
 [ 8.42 ]] 6.156
[[9.438]
 [8.42 ]
 [6.156]] 5.106
[[8.42 ]
 [6.156]
 [5.106]] 6.626
[[6.156]
 [5.106]
 [6.626]

6. Diviser les données en train, valid et test data

In [6]:
X_train, y_train = X[:70], y[:70]
X_valid, y_valid = X[70:80], y[70:80]
X_test, y_test = X[80:], y[80:]

print(X_train.shape, X_valid.shape, X_test.shape)

for i in range(len(X_train)):
 print(X_train[i], y_train[i])

#for i in range(len(X_test)):
# print(X_test[i], y_test[i])

(70, 3, 1) (10, 3, 1) (36, 3, 1)
[[17.41]
 [25.37]
 [30.38]] 36.74
[[25.37]
 [30.38]
 [36.74]] 51.5
[[30.38]
 [36.74]
 [51.5 ]] 69.64
[[36.74]
 [51.5 ]
 [69.64]] 72.08
[[51.5 ]
 [69.64]
 [72.08]] 57.16
[[69.64]
 [72.08]
 [57.16]] 42.24
[[72.08]
 [57.16]
 [42.24]] 34.46
[[57.16]
 [42.24]
 [34.46]] 37.96
[[42.24]
 [34.46]
 [37.96]] 48.31
[[34.46]
 [37.96]
 [48.31]] 58.88
[[37.96]
 [48.31]
 [58.88]] 53.85
[[48.31]
 [58.88]
 [53.85]] 45.15
[[58.88]
 [53.85]
 [45.15]] 37.22
[[53.85]
 [45.15]
 [37.22]] 31.66
[[45.15]
 [37.22]
 [31.66]] 32.68
[[37.22]
 [31.66]
 [32.68]] 42.2
[[31.66]
 [32.68]
 [42.2 ]] 63.83
[[32.68]
 [42.2 ]
 [63.83]] 55.17
[[42.2 ]
 [63.83]
 [55.17]] 38.87
[[63.83]
 [55.17]
 [38.87]] 24.79
[[55.17]
 [38.87]
 [24.79]] 17.85
[[38.87]
 [24.79]
 [17.85]] 15.75
[[24.79]
 [17.85]
 [15.75]] 13.19
[[17.85]
 [15.75]
 [13.19]] 9.438
[[15.75 ]
 [13.19 ]
 [ 9.438]] 8.42
[[13.19 ]
 [ 9.438]
 [ 8.42 ]] 6.156
[[9.438]
 [8.42 ]
 [6.156]] 5.106
[[8.42 ]
 [6.156]
 [5.106]] 6.626
[[6.156]
 [5

7. Définir un premier modèle LSTM univarié

In [8]:
model = Sequential()
model.add(LSTM(150, activation='relu', kernel_initializer="he_normal",input_shape=(n_steps, n_features)))
model.add(Dense(50,activation="relu", kernel_initializer="he_normal"))
model.add(Dropout(0.2))
model.add(Dense(50,activation="relu", kernel_initializer="he_normal"))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

8. Entrainer le modèle

In [9]:
model.fit(X_train, y_train, epochs=200, batch_size=32, verbose=1, validation_data=[X_valid,y_valid])

Epoch 1/200
3/3 [==============================] - 4s 204ms/step - loss: 2838.2251 - val_loss: 10.2471
Epoch 2/200
3/3 [==============================] - 0s 28ms/step - loss: 435.3116 - val_loss: 25.7746
Epoch 3/200
3/3 [==============================] - 0s 28ms/step - loss: 490.1110 - val_loss: 40.7724
Epoch 4/200
3/3 [==============================] - 0s 27ms/step - loss: 541.1368 - val_loss: 39.7068
Epoch 5/200
3/3 [==============================] - 0s 32ms/step - loss: 515.2003 - val_loss: 29.7596
Epoch 6/200
3/3 [==============================] - 0s 29ms/step - loss: 303.1404 - val_loss: 22.6505
Epoch 7/200
3/3 [==============================] - 0s 26ms/step - loss: 394.2332 - val_loss: 19.5781
Epoch 8/200
3/3 [==============================] - 0s 30ms/step - loss: 442.9881 - val_loss: 17.6552
Epoch 9/200
3/3 [==============================] - 0s 36ms/step - loss: 253.8015 - val_loss: 17.0615
Epoch 10/200
3/3 [==============================] - 0s 33ms/step - loss: 289.9391 - val_l

9. Évaluer le modèle

In [10]:
loss  = model.evaluate(X_test, y_test, verbose=0)
print(loss)

100.69373321533203


10. Regarder les prévisions pour les données de tests

In [11]:
y_pred = model.predict(X_test, verbose=0)

for i in range(len(X_test)):
 print(X_test[i], y_test[i], y_pred[i])

[[11.44 ]
 [13.41 ]
 [ 9.968]] 5.758 [7.0750556]
[[13.41 ]
 [ 9.968]
 [ 5.758]] 3.485 [3.3405106]
[[9.968]
 [5.758]
 [3.485]] 2.562 [1.169691]
[[5.758]
 [3.485]
 [2.562]] 2.538 [1.6986424]
[[3.485]
 [2.562]
 [2.538]] 3.102 [2.254063]
[[2.562]
 [2.538]
 [3.102]] 18.4 [3.0376458]
[[ 2.538]
 [ 3.102]
 [18.4  ]] 45.4 [15.784789]
[[ 3.102]
 [18.4  ]
 [45.4  ]] 62.69 [38.56826]
[[18.4 ]
 [45.4 ]
 [62.69]] 55.89 [55.03505]
[[45.4 ]
 [62.69]
 [55.89]] 53.82 [34.745697]
[[62.69]
 [55.89]
 [53.82]] 49.78 [60.21177]
[[55.89]
 [53.82]
 [49.78]] 39.02 [41.141994]
[[53.82]
 [49.78]
 [39.02]] 39.25 [22.331747]
[[49.78]
 [39.02]
 [39.25]] 33.63 [47.08503]
[[39.02]
 [39.25]
 [33.63]] 21.36 [20.625784]
[[39.25]
 [33.63]
 [21.36]] 13.94 [11.424138]
[[33.63]
 [21.36]
 [13.94]] 8.779 [7.934799]
[[21.36 ]
 [13.94 ]
 [ 8.779]] 9.718 [5.1800675]
[[13.94 ]
 [ 8.779]
 [ 9.718]] 10.73 [6.893275]
[[ 8.779]
 [ 9.718]
 [10.73 ]] 9.286 [8.774528]
[[ 9.718]
 [10.73 ]
 [ 9.286]] 7.464 [6.8838]
[[10.73 ]
 [ 9.286]
 [ 7

11. Résumé du modèle

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 150)               91200     
                                                                 
 dense_3 (Dense)             (None, 50)                7550      
                                                                 
 dropout_2 (Dropout)         (None, 50)                0         
                                                                 
 dense_4 (Dense)             (None, 50)                2550      
                                                                 
 dropout_3 (Dropout)         (None, 50)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 51        
                                                                 
Total params: 101351 (395.90 KB)
Trainable params: 101

SECTION 2: Série à input multiple: nous allons maintenant prédire les débits en fonction des débits passés, mais aussi en fonction des températures et précipitations passées (d'où le "input multiple")

12. Assignation des séries à des séquences spécifiques

In [22]:
raw_seq2 = np.array(df)
raw_seq2 = raw_seq2[:,3:]
raw_seq2 = raw_seq2.astype(np.float32)
print(raw_seq2)

[[ 7.2    0.    17.41 ]
 [ 6.4    0.    25.37 ]
 [ 5.7    0.    30.38 ]
 [ 7.6    0.    36.74 ]
 [ 5.9   11.8   51.5  ]
 [ 5.2    0.7   69.64 ]
 [ 4.9    1.7   72.08 ]
 [ 2.6    0.    57.16 ]
 [ 3.5    0.    42.24 ]
 [10.2    0.    34.46 ]
 [ 8.9    9.3   37.96 ]
 [ 5.1    8.9   48.31 ]
 [ 6.2    3.1   58.88 ]
 [ 8.7    3.1   53.85 ]
 [ 6.4    0.    45.15 ]
 [ 8.5    0.    37.22 ]
 [ 9.2    0.2   31.66 ]
 [ 9.    16.7   32.68 ]
 [ 7.2   24.6   42.2  ]
 [ 9.2    0.    63.83 ]
 [ 6.5    0.2   55.17 ]
 [ 8.4    1.4   38.87 ]
 [12.4    0.    24.79 ]
 [15.1    0.    17.85 ]
 [13.     0.    15.75 ]
 [ 7.4    0.    13.19 ]
 [ 7.     0.     9.438]
 [12.3    0.     8.42 ]
 [13.2    0.     6.156]
 [16.2    0.     5.106]
 [ 9.1    0.     6.626]
 [11.4    0.     5.63 ]
 [12.9    0.     4.519]
 [ 5.3    3.1    4.226]
 [ 3.2    0.     4.337]
 [ 5.6    0.     3.58 ]
 [11.     0.     3.411]
 [17.1    4.8    3.469]
 [12.    18.3    6.423]
 [ 6.7    0.     7.25 ]
 [11.7    0.     5.621]
 [10.2    0.7   

13. Fonction servant à formatter les données d'une séquence avec input multiple

In [23]:
def split_sequence2(sequence, n_steps):
 X, y = list(), list()
 for i in range(len(sequence)):
 # find the end of this pattern
  end_ix = i + n_steps
 # check if we are beyond the sequence
  if end_ix > len(sequence)-1:
    break
 # gather input and output parts of the pattern
  seq_x, seq_y = sequence[i:end_ix,:], sequence[end_ix,-1]
  X.append(seq_x)
  y.append(seq_y)
 return array(X), array(y)

14. Formatter les données à l'aide de la fonction précédente

In [24]:
n_steps = 3

X, y = split_sequence2(raw_seq2, n_steps)

n_features = 3
X = X.reshape((X.shape[0], X.shape[1], n_features))

print(X.shape, y.shape)

for i in range(len(X)):
 print(X[i], y[i])

(116, 3, 3) (116,)
[[ 7.2   0.   17.41]
 [ 6.4   0.   25.37]
 [ 5.7   0.   30.38]] 36.74
[[ 6.4   0.   25.37]
 [ 5.7   0.   30.38]
 [ 7.6   0.   36.74]] 51.5
[[ 5.7   0.   30.38]
 [ 7.6   0.   36.74]
 [ 5.9  11.8  51.5 ]] 69.64
[[ 7.6   0.   36.74]
 [ 5.9  11.8  51.5 ]
 [ 5.2   0.7  69.64]] 72.08
[[ 5.9  11.8  51.5 ]
 [ 5.2   0.7  69.64]
 [ 4.9   1.7  72.08]] 57.16
[[ 5.2   0.7  69.64]
 [ 4.9   1.7  72.08]
 [ 2.6   0.   57.16]] 42.24
[[ 4.9   1.7  72.08]
 [ 2.6   0.   57.16]
 [ 3.5   0.   42.24]] 34.46
[[ 2.6   0.   57.16]
 [ 3.5   0.   42.24]
 [10.2   0.   34.46]] 37.96
[[ 3.5   0.   42.24]
 [10.2   0.   34.46]
 [ 8.9   9.3  37.96]] 48.31
[[10.2   0.   34.46]
 [ 8.9   9.3  37.96]
 [ 5.1   8.9  48.31]] 58.88
[[ 8.9   9.3  37.96]
 [ 5.1   8.9  48.31]
 [ 6.2   3.1  58.88]] 53.85
[[ 5.1   8.9  48.31]
 [ 6.2   3.1  58.88]
 [ 8.7   3.1  53.85]] 45.15
[[ 6.2   3.1  58.88]
 [ 8.7   3.1  53.85]
 [ 6.4   0.   45.15]] 37.22
[[ 8.7   3.1  53.85]
 [ 6.4   0.   45.15]
 [ 8.5   0.   37.22]] 31.66
[[

15. Diviser les données en train, valid et test data

In [25]:
X_train, y_train = X[:70], y[:70]
X_valid, y_valid = X[70:80], y[70:80]
X_test, y_test = X[80:], y[80:]

print(X_train.shape, X_valid.shape, X_test.shape)

for i in range(len(X_train)):
 print(X_train[i], y_train[i])

#for i in range(len(X_test)):
# print(X_test[i], y_test[i])

(70, 3, 3) (10, 3, 3) (36, 3, 3)
[[ 7.2   0.   17.41]
 [ 6.4   0.   25.37]
 [ 5.7   0.   30.38]] 36.74
[[ 6.4   0.   25.37]
 [ 5.7   0.   30.38]
 [ 7.6   0.   36.74]] 51.5
[[ 5.7   0.   30.38]
 [ 7.6   0.   36.74]
 [ 5.9  11.8  51.5 ]] 69.64
[[ 7.6   0.   36.74]
 [ 5.9  11.8  51.5 ]
 [ 5.2   0.7  69.64]] 72.08
[[ 5.9  11.8  51.5 ]
 [ 5.2   0.7  69.64]
 [ 4.9   1.7  72.08]] 57.16
[[ 5.2   0.7  69.64]
 [ 4.9   1.7  72.08]
 [ 2.6   0.   57.16]] 42.24
[[ 4.9   1.7  72.08]
 [ 2.6   0.   57.16]
 [ 3.5   0.   42.24]] 34.46
[[ 2.6   0.   57.16]
 [ 3.5   0.   42.24]
 [10.2   0.   34.46]] 37.96
[[ 3.5   0.   42.24]
 [10.2   0.   34.46]
 [ 8.9   9.3  37.96]] 48.31
[[10.2   0.   34.46]
 [ 8.9   9.3  37.96]
 [ 5.1   8.9  48.31]] 58.88
[[ 8.9   9.3  37.96]
 [ 5.1   8.9  48.31]
 [ 6.2   3.1  58.88]] 53.85
[[ 5.1   8.9  48.31]
 [ 6.2   3.1  58.88]
 [ 8.7   3.1  53.85]] 45.15
[[ 6.2   3.1  58.88]
 [ 8.7   3.1  53.85]
 [ 6.4   0.   45.15]] 37.22
[[ 8.7   3.1  53.85]
 [ 6.4   0.   45.15]
 [ 8.5   0.   37

16. Définir le modèle LSTM à input multiple

In [26]:
model = Sequential()
model.add(LSTM(150, activation='relu', kernel_initializer="he_normal",input_shape=(n_steps, n_features)))
model.add(Dense(50,activation="relu", kernel_initializer="he_normal"))
model.add(Dropout(0.2))
model.add(Dense(50,activation="relu", kernel_initializer="he_normal"))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

17. Entrainer le modèle

In [27]:
model.fit(X_train, y_train, epochs=200, batch_size=32, verbose=1, validation_data=[X_valid,y_valid])

Epoch 1/200
3/3 [==============================] - 2s 170ms/step - loss: 5593.2090 - val_loss: 784.9611
Epoch 2/200
3/3 [==============================] - 0s 30ms/step - loss: 2142.7742 - val_loss: 238.0730
Epoch 3/200
3/3 [==============================] - 0s 27ms/step - loss: 1243.1100 - val_loss: 51.6351
Epoch 4/200
3/3 [==============================] - 0s 24ms/step - loss: 674.9398 - val_loss: 25.8151
Epoch 5/200
3/3 [==============================] - 0s 27ms/step - loss: 230.7498 - val_loss: 62.3183
Epoch 6/200
3/3 [==============================] - 0s 25ms/step - loss: 462.5409 - val_loss: 88.1824
Epoch 7/200
3/3 [==============================] - 0s 26ms/step - loss: 275.0625 - val_loss: 81.8810
Epoch 8/200
3/3 [==============================] - 0s 27ms/step - loss: 316.8456 - val_loss: 71.9619
Epoch 9/200
3/3 [==============================] - 0s 23ms/step - loss: 364.7011 - val_loss: 67.4382
Epoch 10/200
3/3 [==============================] - 0s 29ms/step - loss: 191.4506 - v

18. Évaluer le modèle

In [28]:
loss  = model.evaluate(X_test, y_test, verbose=0)
print(loss)

260.5631103515625


19. Regarder les prévisions pour les données de test

In [29]:
y_pred = model.predict(X_test, verbose=0)

for i in range(len(X_test)):
 print(X_test[i], y_test[i], y_pred[i])

[[21.8   19.5   11.44 ]
 [20.8    0.    13.41 ]
 [23.3    0.     9.968]] 5.758 [10.714093]
[[20.8    0.    13.41 ]
 [23.3    0.     9.968]
 [23.9    0.     5.758]] 3.485 [8.1301565]
[[23.3    0.     9.968]
 [23.9    0.     5.758]
 [24.8    0.     3.485]] 2.562 [3.2910817]
[[23.9    0.     5.758]
 [24.8    0.     3.485]
 [22.6   14.5    2.562]] 2.538 [2.9992456]
[[24.8    0.     3.485]
 [22.6   14.5    2.562]
 [22.6    0.     2.538]] 3.102 [4.5462527]
[[22.6   14.5    2.562]
 [22.6    0.     2.538]
 [23.4    7.3    3.102]] 18.4 [1.6626779]
[[22.6    0.     2.538]
 [23.4    7.3    3.102]
 [20.    47.4   18.4  ]] 45.4 [14.025172]
[[23.4    7.3    3.102]
 [20.    47.4   18.4  ]
 [20.    30.9   45.4  ]] 62.69 [32.37211]
[[20.   47.4  18.4 ]
 [20.   30.9  45.4 ]
 [20.3   0.   62.69]] 55.89 [57.39104]
[[20.   30.9  45.4 ]
 [20.3   0.   62.69]
 [18.3  58.1  55.89]] 53.82 [125.25615]
[[20.3   0.   62.69]
 [18.3  58.1  55.89]
 [21.3   0.   53.82]] 49.78 [72.84215]
[[18.3  58.1  55.89]
 [21.3   0

20. Résumé du modèle

In [30]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 150)               92400     
                                                                 
 dense_6 (Dense)             (None, 50)                7550      
                                                                 
 dropout_4 (Dropout)         (None, 50)                0         
                                                                 
 dense_7 (Dense)             (None, 50)                2550      
                                                                 
 dropout_5 (Dropout)         (None, 50)                0         
                                                                 
 dense_8 (Dense)             (None, 1)                 51        
                                                                 
Total params: 102551 (400.59 KB)
Trainable params: 102

SECTION 3: Série multidimensionnelles: nous allons maintenant prédire simultanément les température, précipitations et débits simultanément en fonction des valeurs passés

21. Fonction servant à formatter les données d'une séquence multiple

In [31]:
def split_sequence3(sequence, n_steps):
 X, y = list(), list()
 for i in range(len(sequence)):
 # find the end of this pattern
  end_ix = i + n_steps
 # check if we are beyond the sequence
  if end_ix > len(sequence)-1:
    break
 # gather input and output parts of the pattern
  seq_x, seq_y = sequence[i:end_ix,:], sequence[end_ix,:]
  X.append(seq_x)
  y.append(seq_y)
 return array(X), array(y)

22. Formatter les données à l'aide de la fonction précédente

In [33]:
n_steps = 3

X, y = split_sequence3(raw_seq2, n_steps)

n_features = 3
X = X.reshape((X.shape[0], X.shape[1], n_features))

print(X.shape, y.shape)

for i in range(len(X)):
 print(X[i], y[i])

(116, 3, 3) (116, 3)
[[ 7.2   0.   17.41]
 [ 6.4   0.   25.37]
 [ 5.7   0.   30.38]] [ 7.6   0.   36.74]
[[ 6.4   0.   25.37]
 [ 5.7   0.   30.38]
 [ 7.6   0.   36.74]] [ 5.9 11.8 51.5]
[[ 5.7   0.   30.38]
 [ 7.6   0.   36.74]
 [ 5.9  11.8  51.5 ]] [ 5.2   0.7  69.64]
[[ 7.6   0.   36.74]
 [ 5.9  11.8  51.5 ]
 [ 5.2   0.7  69.64]] [ 4.9   1.7  72.08]
[[ 5.9  11.8  51.5 ]
 [ 5.2   0.7  69.64]
 [ 4.9   1.7  72.08]] [ 2.6   0.   57.16]
[[ 5.2   0.7  69.64]
 [ 4.9   1.7  72.08]
 [ 2.6   0.   57.16]] [ 3.5   0.   42.24]
[[ 4.9   1.7  72.08]
 [ 2.6   0.   57.16]
 [ 3.5   0.   42.24]] [10.2   0.   34.46]
[[ 2.6   0.   57.16]
 [ 3.5   0.   42.24]
 [10.2   0.   34.46]] [ 8.9   9.3  37.96]
[[ 3.5   0.   42.24]
 [10.2   0.   34.46]
 [ 8.9   9.3  37.96]] [ 5.1   8.9  48.31]
[[10.2   0.   34.46]
 [ 8.9   9.3  37.96]
 [ 5.1   8.9  48.31]] [ 6.2   3.1  58.88]
[[ 8.9   9.3  37.96]
 [ 5.1   8.9  48.31]
 [ 6.2   3.1  58.88]] [ 8.7   3.1  53.85]
[[ 5.1   8.9  48.31]
 [ 6.2   3.1  58.88]
 [ 8.7   3.1  53

23. Diviser les données en train, valid et test data

In [34]:
X_train, y_train = X[:70], y[:70]
X_valid, y_valid = X[70:80], y[70:80]
X_test, y_test = X[80:], y[80:]

print(X_train.shape, X_valid.shape, X_test.shape)

for i in range(len(X_train)):
 print(X_train[i], y_train[i])

#for i in range(len(X_test)):
# print(X_test[i], y_test[i])

(70, 3, 3) (10, 3, 3) (36, 3, 3)
[[ 7.2   0.   17.41]
 [ 6.4   0.   25.37]
 [ 5.7   0.   30.38]] [ 7.6   0.   36.74]
[[ 6.4   0.   25.37]
 [ 5.7   0.   30.38]
 [ 7.6   0.   36.74]] [ 5.9 11.8 51.5]
[[ 5.7   0.   30.38]
 [ 7.6   0.   36.74]
 [ 5.9  11.8  51.5 ]] [ 5.2   0.7  69.64]
[[ 7.6   0.   36.74]
 [ 5.9  11.8  51.5 ]
 [ 5.2   0.7  69.64]] [ 4.9   1.7  72.08]
[[ 5.9  11.8  51.5 ]
 [ 5.2   0.7  69.64]
 [ 4.9   1.7  72.08]] [ 2.6   0.   57.16]
[[ 5.2   0.7  69.64]
 [ 4.9   1.7  72.08]
 [ 2.6   0.   57.16]] [ 3.5   0.   42.24]
[[ 4.9   1.7  72.08]
 [ 2.6   0.   57.16]
 [ 3.5   0.   42.24]] [10.2   0.   34.46]
[[ 2.6   0.   57.16]
 [ 3.5   0.   42.24]
 [10.2   0.   34.46]] [ 8.9   9.3  37.96]
[[ 3.5   0.   42.24]
 [10.2   0.   34.46]
 [ 8.9   9.3  37.96]] [ 5.1   8.9  48.31]
[[10.2   0.   34.46]
 [ 8.9   9.3  37.96]
 [ 5.1   8.9  48.31]] [ 6.2   3.1  58.88]
[[ 8.9   9.3  37.96]
 [ 5.1   8.9  48.31]
 [ 6.2   3.1  58.88]] [ 8.7   3.1  53.85]
[[ 5.1   8.9  48.31]
 [ 6.2   3.1  58.88]
 [ 8

24. Définir le modèle LSTM multidimendionnel

In [35]:
model = Sequential()
model.add(LSTM(150, activation='relu', kernel_initializer="he_normal",input_shape=(n_steps, n_features)))
model.add(Dense(50,activation="relu", kernel_initializer="he_normal"))
model.add(Dropout(0.2))
model.add(Dense(50,activation="relu", kernel_initializer="he_normal"))
model.add(Dropout(0.2))
model.add(Dense(n_features)) #ATTENTION: changement ici pour prédire 3 valeurs
model.compile(optimizer='adam', loss='mse')

25. Entrainer le modèle

In [36]:
model.fit(X_train, y_train, epochs=200, batch_size=32, verbose=1, validation_data=[X_valid,y_valid])

Epoch 1/200
3/3 [==============================] - 2s 164ms/step - loss: 1921.1163 - val_loss: 490.1615
Epoch 2/200
3/3 [==============================] - 0s 28ms/step - loss: 1305.4266 - val_loss: 232.8055
Epoch 3/200
3/3 [==============================] - 0s 31ms/step - loss: 640.8924 - val_loss: 127.0215
Epoch 4/200
3/3 [==============================] - 0s 26ms/step - loss: 476.5834 - val_loss: 83.4631
Epoch 5/200
3/3 [==============================] - 0s 31ms/step - loss: 330.0291 - val_loss: 62.6808
Epoch 6/200
3/3 [==============================] - 0s 25ms/step - loss: 307.3355 - val_loss: 52.3038
Epoch 7/200
3/3 [==============================] - 0s 35ms/step - loss: 294.0230 - val_loss: 46.5662
Epoch 8/200
3/3 [==============================] - 0s 26ms/step - loss: 257.2898 - val_loss: 42.9985
Epoch 9/200
3/3 [==============================] - 0s 28ms/step - loss: 264.4865 - val_loss: 40.8326
Epoch 10/200
3/3 [==============================] - 0s 24ms/step - loss: 201.7749 - v

26. Évaluer le modèle sur les données de test

In [37]:
loss  = model.evaluate(X_test, y_test, verbose=0)
print(loss)

196.72198486328125


27. Regarder les prévisions sur les données de test

In [38]:
y_pred = model.predict(X_test, verbose=0)

for i in range(len(X_test)):
 print(X_test[i], y_test[i], y_pred[i])

[[21.8   19.5   11.44 ]
 [20.8    0.    13.41 ]
 [23.3    0.     9.968]] [23.9    0.     5.758] [11.085297   -0.24132307  5.764153  ]
[[20.8    0.    13.41 ]
 [23.3    0.     9.968]
 [23.9    0.     5.758]] [24.8    0.     3.485] [14.289177   -0.15459812  3.7340105 ]
[[23.3    0.     9.968]
 [23.9    0.     5.758]
 [24.8    0.     3.485]] [22.6   14.5    2.562] [23.330206   -0.21951516  5.0423822 ]
[[23.9    0.     5.758]
 [24.8    0.     3.485]
 [22.6   14.5    2.562]] [22.6    0.     2.538] [15.139746   2.0697207  2.5483274]
[[24.8    0.     3.485]
 [22.6   14.5    2.562]
 [22.6    0.     2.538]] [23.4    7.3    3.102] [17.459303  13.93686    1.2016383]
[[22.6   14.5    2.562]
 [22.6    0.     2.538]
 [23.4    7.3    3.102]] [20.  47.4 18.4] [22.18488    3.6268375  2.5329344]
[[22.6    0.     2.538]
 [23.4    7.3    3.102]
 [20.    47.4   18.4  ]] [20.  30.9 45.4] [12.543207 10.439823  9.130147]
[[23.4    7.3    3.102]
 [20.    47.4   18.4  ]
 [20.    30.9   45.4  ]] [20.3   0.   62.

28. Résumé du modèle

In [39]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 150)               92400     
                                                                 
 dense_9 (Dense)             (None, 50)                7550      
                                                                 
 dropout_6 (Dropout)         (None, 50)                0         
                                                                 
 dense_10 (Dense)            (None, 50)                2550      
                                                                 
 dropout_7 (Dropout)         (None, 50)                0         
                                                                 
 dense_11 (Dense)            (None, 3)                 153       
                                                                 
Total params: 102653 (400.99 KB)
Trainable params: 102